In [1]:
#export
import torch.nn as nn
import k1lib, re

In [2]:
#export
def getParts(s): return [a for elem in s.split(":")[0].split(">") if elem for a in elem.split(" ") if a]
def getProps(s):
    splits = s.split(":")
    if len(splits) == 1: return ["all"]
    return [elem for elem in splits[1].split(",") if elem]
class ModuleSelector:
    def __init__(self, parent, name:str, nnModule:nn.Module):
        self.parent = parent; self.name = name; self.nnModule = nnModule
        self._children = []; self.directSelectors = []; self.indirectSelectors = []
        self.selectedProps = []
    def parse(self, selectors):
        ogSelectors = selectors
        if self.parent != None:
            selectors = [] + selectors + self.parent.indirectSelectors + self.parent.directSelectors
            self.indirectSelectors += self.parent.indirectSelectors
        for selector in selectors:
            if selector == "all": self.selectedProps = ["all"]; break
            parts = getParts(selector)
            matches = parts[0] == self.nnModule.__class__.__name__ or parts[0] == "#" + self.name or parts[0] == "*"
            if len(parts) == 1:
                if matches: self.selectedProps = getProps(selector)
            else:
                a = selector.find(">"); a = a if a > 0 else float("inf")
                b = selector.find(" "); b = b if b > 0 else float("inf")
                direct = a < b
                if matches:
                    if direct: self.directSelectors.append(selector[a+1:])
                    else: self.indirectSelectors.append(selector[b+1:])
        for name, mod in self.nnModule.named_children():
            self._children.append(child := ModuleSelector(self, name, mod))
            child.parse(ogSelectors)
        self._children = {child.name: child for child in self._children}
    def selected(self, prop="_propShouldNotExist"):
        return "all" in self.selectedProps or prop in self.selectedProps
    def named_children(self): return self._children.items()
    def named_modules(self):
        yield self.name, self
        for child in self._children.values():
            for x in child.named_modules(): yield x
    def children(self):
        for name, x in self.named_children(): yield x
    def modules(self):
        for name, x in self.named_modules(): yield x
    def __getattr__(self, attr): return self._children[attr]
    def __getitem__(self, idx): return self._children[str(idx)]
    def __repr__(self, intro=True):
        answer = "ModuleSelector:\n" if intro else ""
        answer += f"{self.name}: {self.nnModule.__class__.__name__}                     {'' if len(self.selectedProps) == 0 else ', '.join(self.selectedProps)}"
        if len(self._children) > 0: answer += "\n"
        answer += k1lib.tab("\n".join([child.__repr__(intro=False) for name, child in self._children.items()]))
        return answer
def select(model:nn.Module, selectors):
    root = ModuleSelector(None, "root", model)
    root.parse([re.sub("\s\s+", " ", line).replace(" >", ">").replace("> ", ">").replace(" :", ":").replace(": ", ":").replace(" ,", ",").replace(", ", ",").replace(" \n", "\n").replace("\n ", "\n") for line in selectors.split("\n") if line != ""])
    return root

In [3]:
!../export.py _selectors

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> bui